In [1]:
import numpy as np
import numpy.ma as ma
import os
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import itertools
from matplotlib import gridspec

import sys

In [2]:
DATA_DIR = "/scratch9/mkron/data/NN_out/run23/"
predictions_mctrue = pickle.load(file(os.path.join(DATA_DIR, 'best_val_loss_pred.pickle')))
epochs_acc_loss = np.delete(np.genfromtxt(os.path.join(DATA_DIR, "loss_logger.csv"), delimiter=','), 0, axis=0).T

In [3]:
def retag_classify_startTrack(ClassLabel):
    ClassLabelRetag = ClassLabel
    ClassLabelRetag[ClassLabel == 0] =0
    ClassLabelRetag[ClassLabel == 1] =0
    ClassLabelRetag[ClassLabel == 2] =1
    ClassLabelRetag[ClassLabel == 3] =3
    ClassLabelRetag[ClassLabel == 4] =1
    ClassLabelRetag[ClassLabel == 5] =2
    ClassLabelRetag[ClassLabel == 6] =2
    ClassLabelRetag[ClassLabel == 7] =0
    ClassLabelRetag[ClassLabel == 8] =1
    ClassLabelRetag[ClassLabel == 9] =0
    return ClassLabelRetag

In [4]:
### ACHTUNG nur einmalig ausführen

pred_label_1 = predictions_mctrue["prediction"][0] #list with probability for each class
true_label_1_raw = predictions_mctrue["mc_truth"]["ClassificationLabel_truth"][:len(pred_label_1)]
# ReTag ClassLabel to Labels which can be predicted
true_label_1 = retag_classify_startTrack(true_label_1_raw)

#The actuall Predictions for class 1
all_pred = np.argmax(pred_label_1, axis=1) # all predictions

#Energy
energy=[]
for i in xrange(0, len(pred_label_1)):
    energy.append(predictions_mctrue["reco_vals"][i][0])
energy = np.array(energy)

# Number of hit DOMs
hitDOMs = predictions_mctrue["HitDOMs"][:len(pred_label_1)]
hitDOMs = np.array(hitDOMs)

#RUN and EVENT-ID and primary particle
RunID = []
EventID = []
Primary_pdg = []
for i in xrange(len(pred_label_1)):
    RunID.append(predictions_mctrue["reco_vals"][i][5])
    EventID.append(predictions_mctrue["reco_vals"][i][6])
    Primary_pdg.append(predictions_mctrue["reco_vals"][i][2])
RunID = np.array(RunID)
EventID = np.array(EventID)
Primary_pdg = np.array(Primary_pdg)

In [5]:
print true_label_1[hitDOMs == np.min(hitDOMs)]
print all_pred[hitDOMs == np.min(hitDOMs)]
print RunID[hitDOMs == np.min(hitDOMs)]
print EventID[hitDOMs == np.min(hitDOMs)]

[ 0.  0.  0.]
[0 0 0]
[ 19733.  19255.  22850.]
[ 7984.  6637.  6382.]


In [6]:
print true_label_1[hitDOMs == np.max(hitDOMs)]
print all_pred[hitDOMs == np.max(hitDOMs)]
print RunID[hitDOMs == np.max(hitDOMs)]
print EventID[hitDOMs == np.max(hitDOMs)]

[ 1.]
[1]
[ 22748.]
[ 7532.]


In [7]:
true_bed = true_label_1[hitDOMs == 100]
pred_bed = all_pred[hitDOMs == 100]
RID_bed = RunID[hitDOMs == 100]
EID_bed = EventID[hitDOMs == 100]
prim_bed = Primary_pdg[hitDOMs == 100]


print true_bed[pred_bed != true_bed]
print pred_bed[pred_bed != true_bed]
print prim_bed[pred_bed != true_bed]
print RID_bed[pred_bed != true_bed]
print EID_bed[pred_bed != true_bed]

[ 0.  3.  1.  3.  3.  2.  3.  1.  3.  3.  3.  1.  1.  3.  0.  3.  0.  0.
  1.]
[1 1 0 0 0 0 0 0 1 0 1 3 0 0 1 0 3 3 3]
[ 16.  14.  14. -14.  14. -16.  14.  14.  14. -14. -14.  14.  16. -16.  12.
  14.  16. -12. -14.]
[ 19833.  19836.  19302.  22670.  19359.  19288.   9378.  19011.  22395.
  19004.  19004.  19004.  19031.   9281.  22898.  19160.  19704.  19205.
  19812.]
[ 1472.    46.  9249.  6139.  2991.  8991.  6280.  9170.  8376.  1701.
  1021.  9872.  5342.  4992.  2443.  3399.  8048.   680.   283.]


### Enter RunID and EventID -> get Infos to this event

In [116]:
RunIDN = 9281
EventIDN = 9447

eventindex = np.where(EventID == EventIDN) 
#print eventindex[0][2]
runindex = np.where(RunID == RunIDN)
#print runindex[0]

for i in xrange(len(eventindex[0])):
    index = np.where(runindex[0] == eventindex[0][i])
    if index[0] !=0:
        break
#print index
a =  runindex[0][index[0]][0]
#print a
the_event = predictions_mctrue["reco_vals"][a]

print "Primary: {}".format(np.abs(the_event[2]))
print "True Event Type: {}".format(true_label_1[a])
print "Pred Event Type: {}".format(all_pred[a])
print "Number of hit DOMs: {}".format(hitDOMs[a])
print "Detailled Prediction {}".format(' & '.join(['{:.2f}'.format(x*100) for x in pred_label_1[a]]))

#print the_event



Primary: 16.0
True Event Type: 2.0
Pred Event Type: 3
Number of hit DOMs: 1097
Detailled Prediction 1.01 & 19.52 & 2.00 & 77.47
